Utilities:

In [1]:
from collections import deque
import sys
sys.path.append('..')
from shared.aocutil import slice_deque

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
[4, 5, 6, 7, 8, 9, 10]
deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])


Solution:

In [2]:
crt_width = 40

def read_instr(file) -> list:
    return file.readline().split()


def delay_instr(instruction: str):
    """ Returns a generator function that waits the proper number of cycles before letting the instruction execute """
    match instruction:
        case ['noop']:
            delayed = (_ for _ in [True])
        case ['addx', *others]:
            delayed = (_ for _ in (False, True))
        case [cmd, *others]:
                raise ValueError(f'{cmd} with args {others} not supported.')
    return delayed


def draw_sprite(reg_x: int, width: int) -> list:
    pos = [p for p in [reg_x - 1, reg_x, reg_x + 1] if 0 <= p < width]
    return pos


def draw_pixel(cycle: int, sprite_positions: list) -> str:
    # Don't forget that cycles are 1-indexed and pixels are 0-indexed!
    return '#' if (cycle - 1) % crt_width in sprite_positions else '.'


with open('../inputs/day10-input') as f:

    """ This loop is the CPU """
    # Init state
    cycle = 1  #  Required for 'signal_strength' math
    signal_strengths = []
    reg_x = 1
    sprite_pos = draw_sprite(reg_x, crt_width)
    crt_buffer = deque()

    # Load first instruction
    instr = read_instr(f)
    delayed_exec = delay_instr(instr)

    while True:
        """
        CPU operation order (loop starts at 4):
        1. Update clock cycle
        2. If last instruction complete, read another
        3. If objectives are met or instruction list ended, quit
        4A (Pt. 1). Measure signal strength
        4B (Pt. 2). Draw a pixel
        5. Execute current instruction (might be delayed/continued from last cycle)
        5A (Pt. 2). Update the sprite position
        """
        print(f'{cycle}: X = {reg_x} // {" ".join(instr)}')  # debug

        if len(signal_strengths) < 6:
            if (cycle - 20) % 40 == 0:
                signal_strengths.append(cycle * reg_x)
                print(f'Signal strength = {signal_strengths[-1]}')  # debug

        crt_buffer.append(draw_pixel(cycle, sprite_pos))

        # --- Clock Cycle Ends ---
        if next(delayed_exec):
            match instr:
                case ['noop']:
                    done = True
                case ['addx', value]:
                    reg_x += int(value)
                    sprite_pos = draw_sprite(reg_x, crt_width)
                    done = True
                case [cmd, *others]:
                    raise ValueError(f'{cmd} with args {others} not supported.')

        # --- Cycle Begins
        cycle += 1

        if done:
            done = False
            instr = read_instr(f)

            if instr:
                delayed_exec = delay_instr(instr)
            else:
                break

print(f'Signal strengths: {signal_strengths} // Total: {sum(signal_strengths)}')

for r in range(len(crt_buffer) // crt_width):
    crt_line = slice_deque(crt_buffer, crt_width * r, crt_width * r + crt_width)
    print(''.join(crt_line))

1: X = 1 // noop
2: X = 1 // addx 5
3: X = 1 // addx 5
4: X = 6 // noop
5: X = 6 // noop
6: X = 6 // noop
7: X = 6 // addx 1
8: X = 6 // addx 1
9: X = 7 // addx 2
10: X = 7 // addx 2
11: X = 9 // addx 5
12: X = 9 // addx 5
13: X = 14 // addx 2
14: X = 14 // addx 2
15: X = 16 // addx 5
16: X = 16 // addx 5
17: X = 21 // noop
18: X = 21 // noop
19: X = 21 // noop
20: X = 21 // noop
Signal strength = 420
21: X = 21 // noop
22: X = 21 // addx -12
23: X = 21 // addx -12
24: X = 9 // addx 18
25: X = 9 // addx 18
26: X = 27 // addx -1
27: X = 27 // addx -1
28: X = 26 // noop
29: X = 26 // addx 3
30: X = 26 // addx 3
31: X = 29 // addx 5
32: X = 29 // addx 5
33: X = 34 // addx -5
34: X = 34 // addx -5
35: X = 29 // addx 7
36: X = 29 // addx 7
37: X = 36 // noop
38: X = 36 // addx -36
39: X = 36 // addx -36
40: X = 0 // addx 18
41: X = 0 // addx 18
42: X = 18 // addx -16
43: X = 18 // addx -16
44: X = 2 // noop
45: X = 2 // noop
46: X = 2 // noop
47: X = 2 // addx 5
48: X = 2 // addx 5
49: X = 